In [1]:
import os
import math

import numpy as np
import tensorflow as tf

slim = tf.contrib.slim

In [2]:
import sys
sys.path.append('../')

In [3]:
from models_slim import inception, inception_utils
from models_slim import inception_v1, inception_v2, inception_v3, inception_v4

In [4]:
# TensorFlow session: grow memory when needed. TF, DO NOT USE ALL MY GPU MEMORY!!!
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(log_device_placement=False, gpu_options=gpu_options)
isess = tf.InteractiveSession(config=config)

# Inception checkpoints

Add scale parameter to be able to use fused-BN.

In [5]:
def restore_checkpoint(sess, ckpt_filename, moving_average_decay=0.9999):
    tf_global_step = slim.get_or_create_global_step()
    # Restore moving average variables or classic stuff!
    if moving_average_decay:
        variable_averages = tf.train.ExponentialMovingAverage(
            moving_average_decay, tf_global_step)
        variables_to_restore = variable_averages.variables_to_restore(
            tf.contrib.framework.get_model_variables())
        variables_to_restore[tf_global_step.op.name] = tf_global_step
    else:
        variables_to_restore = tf.contrib.framework.get_variables_to_restore()
    # Restore method.
    fn_restore = slim.assign_from_checkpoint_fn(ckpt_filename,
                                                variables_to_restore,
                                                ignore_missing_vars=True)
    fn_restore(sess)

In [6]:
inception_model = inception.Inceptionv4Model()
ckpt_filename = '../checkpoints/inception_v4.ckpt'
        
with tf.Graph().as_default():
    # Input placeholder.
    data_format = 'NCHW'
    imgsize = inception_model.image_size
    net_shape = (imgsize, imgsize)
    img_input = tf.placeholder(tf.float32, shape=(1, 3, imgsize, imgsize))
    
    # Network...
    output = inception_model.inference(img_input, num_classes=1001, is_training=False, 
                                       data_format=data_format, data_type=tf.float32)
    
    with tf.Session() as sess:
        # Initialize variables.
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        # Load checkpoint. Should WARNINGS because of missing scale parameter...
        restore_checkpoint(sess, ckpt_filename, moving_average_decay=None)
        # Save back checkpoint!
        save_filename = ckpt_filename.replace('.ckpt', '_fused.ckpt')
        saver = tf.train.Saver()
        saver.save(sess, save_filename, write_meta_graph=False)

KeyboardInterrupt: 

# MobileNets - Google

Convert checkpoint from Google format.

In [7]:
from models_slim import mobilenets

In [8]:
# Build up mapping between Google MobileNets and ours.
ckpt_mapping = {
    'MobilenetV1': 'MobileNets',
    'Conv2d_0': 'conv1',
    'Logits/Conv2d_1c_1x1': 'conv_fc15',
}
# Main blocks layers.
for i in range(1, 14):
    ckpt_mapping['Conv2d_%i_depthwise' % i] = 'block%i/conv_dw' % (i+1)
    ckpt_mapping['Conv2d_%i_pointwise' % i] = 'block%i/conv_pw' % (i+1)

ckpt_mapping

{'Conv2d_0': 'conv1',
 'Conv2d_10_depthwise': 'block11/conv_dw',
 'Conv2d_10_pointwise': 'block11/conv_pw',
 'Conv2d_11_depthwise': 'block12/conv_dw',
 'Conv2d_11_pointwise': 'block12/conv_pw',
 'Conv2d_12_depthwise': 'block13/conv_dw',
 'Conv2d_12_pointwise': 'block13/conv_pw',
 'Conv2d_13_depthwise': 'block14/conv_dw',
 'Conv2d_13_pointwise': 'block14/conv_pw',
 'Conv2d_1_depthwise': 'block2/conv_dw',
 'Conv2d_1_pointwise': 'block2/conv_pw',
 'Conv2d_2_depthwise': 'block3/conv_dw',
 'Conv2d_2_pointwise': 'block3/conv_pw',
 'Conv2d_3_depthwise': 'block4/conv_dw',
 'Conv2d_3_pointwise': 'block4/conv_pw',
 'Conv2d_4_depthwise': 'block5/conv_dw',
 'Conv2d_4_pointwise': 'block5/conv_pw',
 'Conv2d_5_depthwise': 'block6/conv_dw',
 'Conv2d_5_pointwise': 'block6/conv_pw',
 'Conv2d_6_depthwise': 'block7/conv_dw',
 'Conv2d_6_pointwise': 'block7/conv_pw',
 'Conv2d_7_depthwise': 'block8/conv_dw',
 'Conv2d_7_pointwise': 'block8/conv_pw',
 'Conv2d_8_depthwise': 'block9/conv_dw',
 'Conv2d_8_pointwis

In [12]:
def restore_checkpoint(sess, ckpt_filename, dict_mapping=None, moving_average_decay=0.9999):
    tf_global_step = slim.get_or_create_global_step()
    # Restore moving average variables or classic stuff!
    if moving_average_decay:
        variable_averages = tf.train.ExponentialMovingAverage(
            moving_average_decay, tf_global_step)
        variables_to_restore = variable_averages.variables_to_restore(
            tf.contrib.framework.get_model_variables())
        variables_to_restore[tf_global_step.op.name] = tf_global_step
    else:
        variables_to_restore = tf.contrib.framework.get_variables_to_restore()
    # Modify var names using mapping dict.
    for a, b in dict_mapping.items():
        variables_to_restore = {k.replace(b, a): v for k, v in variables_to_restore.items()}
    # Restore method.
    fn_restore = slim.assign_from_checkpoint_fn(ckpt_filename,
                                                variables_to_restore,
                                                ignore_missing_vars=True)
    fn_restore(sess)

In [14]:
mobilenets_model = mobilenets.MobileNetsModel()
ckpt_filename = '../checkpoints/mobilenet_v1_1.0_224.ckpt'
        
with tf.Graph().as_default():
    # Input placeholder.
    data_format = 'NCHW'
    imgsize = mobilenets_model.image_size
    net_shape = (imgsize, imgsize)
    img_input = tf.placeholder(tf.float32, shape=(1, 3, imgsize, imgsize))
    
    # Network...
    output = mobilenets_model.inference(img_input, num_classes=1001, is_training=False, 
                                        data_format=data_format, data_type=tf.float32)
    
    with tf.Session() as sess:
        # Initialize variables.
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        # Load checkpoint. Should WARNINGS because of missing scale parameter...
        restore_checkpoint(sess, ckpt_filename, ckpt_mapping, moving_average_decay=0.999)
        # Save back checkpoint!
        save_filename = ckpt_filename.replace('.ckpt', '_slim.ckpt')
        saver = tf.train.Saver()
        saver.save(sess, save_filename, write_meta_graph=True)

INFO:tensorflow:Restoring parameters from ../checkpoints/mobilenet_v1_1.0_224.ckpt
